In [5]:
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [6]:
import requests
import os
import csv
from langchain_teddynote.prompts import load_prompt
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

api_key = os.getenv("UPSTAGE_API_KEY")

url = "https://api.upstage.ai/v1/document-ai/ocr"
headers = {"Authorization": f"Bearer {api_key}"}

image_folder = "image"
image_files = [f for f in os.listdir(image_folder) if f.endswith((".jpg", ".png"))]

prompt = load_prompt("prompt/ocr06.yaml", encoding="utf-8")

# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0.0)

# 단계 8: 체인(Chain) 생성
chain = prompt | llm | StrOutputParser()

In [8]:
for filename in image_files:
    # Open and send each image file to the OCR API
    with open(os.path.join(image_folder, filename), "rb") as file:
        files = {"document": file}
        response = requests.post(url, headers=headers, files=files)
        response_data = response.json()

    # Extract text from response
    pages = response_data.get("pages", [])
    if not pages:
        print(f"OCR 결과에 'pages' 데이터가 없습니다. 파일: {filename}")
        continue  # pages가 비어있으면 해당 파일 건너뛰기

    texts = [page.get("text", "") for page in pages]
    print(f"OCR 결과: {texts}")
    if not texts or not texts[0]:
        print(f"OCR 결과에서 텍스트를 추출하지 못했습니다. 파일: {filename}")
        continue  # 텍스트가 비어있을 경우 건너뛰기
    data = texts[0].replace(",", "")

    # Process the result with the language model
    result = chain.invoke(
        {"context": data, "question": "context를 csv 형식으로 변환해줘"}
    )
    print(f"처리 결과 : {result}")

    # Save the result to a CSV file
    csv_filename = f"{os.path.splitext(filename)[0]}.csv"
    csv_data = result.split("\n")

    filtered_data = []
    record = False
    for line in csv_data:
        if '"""' in line or "```" in line:
            continue
        if "판매 코드" in line:
            record = True
        if record:
            filtered_data.append(line)

    with open(csv_filename, mode="w", newline="", encoding="utf-8-sig") as file:
        writer = csv.writer(file)
        for row in filtered_data:
            writer.writerow(row.split(","))

    print(f"CSV 파일이 {csv_filename}로 저장되었습니다.")

OCR 결과: ['8801046394649 \nPromotion 정보 \n8801046394656 바이컬러 데즐링 블루 칫솔 \n2개 \n8801046394663 바이컬러 헬시온 옐로우 칫솔 5,000 5,000 \n8801046394199 바이컬러 치어리 레드 칫솔 5,000 5,000 할인 애경 2.5만원 이상 구매시 5천원 상품권 증정 \n1EA \n8801046412077 바이컬러 치어리 가글 1EA 5,000 5,000 애경 2.5만원 이상 구매시 5천원 상품권 증정 \n2개 이상 구매시 50% \n1EA \n8801046431849 마우스 스프레이 12,900 12,900 애경 2.5만원 이상 구매시 5천원 상품권 증정 \n2개 이상 구매시 50% 할인 \n바이컬러 \n2개 이상 구매시 50% 할인 \n8801046431856 가글 마일드 8,900 7,900 2.5만원 이상 구매시 5천원 상품권 증정 \n283ML \n진지발리스 \n2개 이상 구매시 50% 할인 애경 \n10ML \n8809949550605 가글 스트롱 5,900 4,900 이상 구매시 5천원 상품권 증정 \n진지발리스 \n800ML \n애경 2.5만원 \n8809949550599 칫솔 77 밸류팩 4,900 구매시 5천원 상품권 증정 \n죽염 8열광폭 \n800ML 5,900 \n애경 2.5만원 이상 \n8809949549944 칫솔 77 밸류팩 6,900 5천원 상품권 증정 \n페리오 토탈7 \n5입 8,900 \n애경 2.5만원 이상 구매시 \n8809949549524 치약 77 밸류팩 행사상품 3만원이상 구매시 5천원 상품권 증정 \n페리오 토탈7 \nLG 행사상품 3만원이상 구매시 5천원 상품권 증정 \n7입 8,900 6,900 \n죽염 은강고 치약 77 밸류팩 LG 행사상품 3만원이상 구매시 5천원 상품권 증정 \n8809949556973 \nLG \n150G*4 8,900 6,900 \n히말라야핑크솔트 바이옴치약 대용량 160G*4 9,900 7,700 \n88099